<a href="https://colab.research.google.com/github/NoufAlshenaifi/Rasid-at-StanceEval2024/blob/main/ML_for_Stance_Detection_(Ensemble_of(SVC%2CMNB%2CLR)).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
########### Ensamble model ###############
# Building voting classifier (linearsvc, multinomialNB, Logistic regression)

In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re # for regular expression
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from collections import Counter

#ML Classification
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics  import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import pickle
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline,FeatureUnion
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.ensemble import VotingClassifier
import torch

In [4]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [112]:
data_df= pd.read_csv('V2_Mawqif_AllTargets_Train.csv')

In [113]:
data_df.head()

,ID,text,target,stance
0,1,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Women empowerment,Against
1,3,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,Covid Vaccine,Nothing
2,4,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Women empowerment,Favor
3,6,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Digital Transformation,Favor
4,7,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Digital Transformation,Favor


In [ ]:
#def build_estimators():
#    estimators = []
#    svc = LinearSVC(penalty='l1', dual=False,tol=1e-3)
#    estimators.append(('svc',svc))
#    mnb= MultinomialNB(alpha=.01)
#    estimators.append(('mnb',mnb))
#    bnb= BernoulliNB(alpha=.01)
#    estimators.append(('bnb',bnb))
#    ensemble = VotingClassifier(estimators)
#    return ensemble

In [114]:
def build_estimators():
    # Initialize a list to hold the tuple pairs of estimator names and their instances
    estimators = []

    # Create a LinearSVC classifier
    svc = LinearSVC(penalty='l1', dual=False, tol=1e-3)
    # Append the LinearSVC classifier to the list of estimators
    estimators.append(('svc', svc))

    # Create a MultinomialNB classifier
    mnb = MultinomialNB(alpha=.01)
    # Append the MultinomialNB classifier to the list of estimators
    estimators.append(('mnb', mnb))

    # Create a Logistic Regression classifier
    logreg = LogisticRegression(max_iter=1000)
    # Append the Logistic Regression classifier to the list of estimators
    estimators.append(('logreg', logreg))

    # Create a voting classifier that combines the individual estimators
    ensemble = VotingClassifier(estimators=estimators)

    # Return the assembled voting classifier
    return ensemble

In [20]:
# Preprocessing ..
import re # for regular expression
# remove punctuations علامات الترقيم
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [21]:
#processing the tweet
def processPost(tweet):
    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', '', tweet)

    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)

    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)


    #remove numbers
    tweet = ''.join(i for i in tweet if not i.isdigit())

    #remove english letters
    tweet= re.sub(r'[a-z]+'," ", tweet)
    tweet= re.sub(r'[A-Z]+'," ", tweet)

    return tweet

In [22]:
data_df["cleanText"] = data_df['text'].apply(lambda x: processPost(x))

In [23]:
data_df.head()

,ID,text,target,stance,cleanText
0,1,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...,Women empowerment,Against,عشان يلمع صورته ويعنني تمكين المرأة ويصير ترن...
1,3,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...,Covid Vaccine,Nothing,روح حلل محد يم تطعيم كورونا شف الحرم البارح م...
2,4,هذا ما يُعرّف بـ'فوبيا المرأة المُتمكنة' آفة ف...,Women empowerment,Favor,هذا ما يُعرّف بفوبيا المرأة المُتمكنة آفة فكري...
3,6,#LEAP22 مؤتمر يجمع اشهر وابرز المؤثرين في الم...,Digital Transformation,Favor,مؤتمر يجمع اشهر وابرز المؤثرين في المجال ال...
4,7,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...,Digital Transformation,Favor,خصوصية البيانات وحمايتها في المنظمة مطلب ولكن ...


In [115]:
x=data_df['text']  + data_df['target']
#x=data_df['cleanText']
y=data_df['stance']

In [116]:
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.1,stratify=y,random_state=42)
#X_train1, X_valid, y_train1, y_valid=train_test_split(X_train, y_train,test_size=0.1,
#                                                      stratify=y_train,random_state=42)

In [117]:
target_names=list(y_train.unique())

In [118]:
target_names

['Against', 'Favor', 'Nothing']

In [ ]:
# featureunion -> combines several transformer objects into a new transformer that combines their output.
# A FeatureUnion takes a list of transformer objects. During fitting, each of these is fit to the data independently.
# For transforming data, the transformers are applied in parallel, and
# the sample vectors they output are concatenated end-to-end into larger vectors.

In [ ]:
#Extracting features from the training data using a Tfidf-Vectorizer

In [119]:
# featureunion -> concatenate the results of multiple transformer objects,combine several feature extraction mechanisms
#into a single transformer. # 1 2 # 2 5
max_df = 0.5
union = FeatureUnion([("w_v", TfidfVectorizer(sublinear_tf=True, max_df=max_df,analyzer = 'word', ngram_range=(1,2)
                                             )),

                      ("c_wb", TfidfVectorizer(sublinear_tf=True,max_df=max_df,analyzer = 'char_wb', ngram_range=(2,4)
                              )),

              #        ("c_wb5", TfidfVectorizer(sublinear_tf=True, max_df=max_df,analyzer = 'char', ngram_range=(2,5)
              #                   )),


                       ],
transformer_weights={
            'w_v': 0.9,
            'c_wb': 0.9,
          #  "c_wb5":0.9,

        }
,
)

X_train_feat = union.fit_transform(X_train)
X_valid_feat = union.transform(X_test)
sm = SMOTE(random_state=42)              ## to solve the imbalanced data problem
X_res, y_res = sm.fit_resample(X_train_feat, y_train)
#X_res=X_train_feat
#y_res=y_train
print("Combined space has", X_res.shape[1], "features")

Combined space has 155365 features


In [12]:
# featureunion -> concatenate the results of multiple transformer objects,combine several feature extraction mechanisms
#into a single transformer.
max_df = 0.5
union = FeatureUnion([("w_v", TfidfVectorizer(sublinear_tf=True, max_df=max_df,analyzer = 'word', ngram_range=(1,3)
                                             )),

                      ("c_wb", TfidfVectorizer(sublinear_tf=True,max_df=max_df,analyzer = 'char_wb', ngram_range=(2,5)
                              )),

                      ("c_wb5", TfidfVectorizer(sublinear_tf=True, max_df=max_df,analyzer = 'char', ngram_range=(2,5)
                                 )),


                       ],
transformer_weights={
            'w_v': 0.9,
            'c_wb': 0.9,
            "c_wb5":0.9,

        }
,
)

X_train_feat = union.fit_transform(X_train)
X_valid_feat = union.transform(X_test)
sm = SMOTE(random_state=42)              ## to solve the imbalanced data problem
X_res, y_res = sm.fit_resample(X_train_feat, y_train)
print("Combined space has", X_res.shape[1], "features")

Combined space has 473583 features


In [120]:
#Training the classifier
ensemble = build_estimators()
ensemble.fit(X_res, y_res)

VotingClassifier(estimators=[('svc',
                              LinearSVC(dual=False, penalty='l1', tol=0.001)),
                             ('mnb', MultinomialNB(alpha=0.01)),
                             ('logreg', LogisticRegression(max_iter=1000))])

In [123]:
#Testing the classifier on validation data
pred = ensemble.predict(X_valid_feat)
#for i in range(0,10):
 #   print(data_train.target_names[pred[i]])

score = metrics.accuracy_score(y_test, pred)
print("Test accuracy:   %0.3f" % score)

print("classification report:")
print(metrics.classification_report(y_test, pred,target_names=target_names))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, pred))

Test accuracy:   0.738
classification report:
              precision    recall  f1-score   support

     Against       0.64      0.73      0.68       102
       Favor       0.80      0.81      0.81       215
     Nothing       0.59      0.29      0.39        34

    accuracy                           0.74       351
   macro avg       0.68      0.61      0.63       351
weighted avg       0.73      0.74      0.73       351

confusion matrix:
[[ 74  26   2]
 [ 35 175   5]
 [  6  18  10]]


In [124]:
from sklearn import metrics
from sklearn.metrics import f1_score
# Predicting on training data to compute training scores
train_pred = ensemble.predict(X_train_feat)
train_f1_macro = f1_score(y_train, train_pred, average='macro')
print("Macro-F1 Train: %0.3f" % train_f1_macro)

# Testing the classifier on validation data
pred = ensemble.predict(X_valid_feat)
test_f1_macro = f1_score(y_test, pred, average='macro')
print("Macro-F1 Test: %0.3f" % test_f1_macro)

Macro-F1 Train: 0.987
Macro-F1 Test: 0.627


In [ ]:
#Macro-F1 Test: 0.63

In [ ]:
# Get prediction ...

In [125]:
blind_data = pd.read_csv('Mawqif_AllTargets_Blind Test.csv')

In [126]:
x_blind = blind_data['text']

In [127]:
# Transform the blind test data using the same union setup
X_blind_feat = union.transform(x_blind)

In [128]:
# Predict using the trained ensemble model
blind_predictions = ensemble.predict(X_blind_feat)

In [129]:
# Attach predictions to the blind data DataFrame and save or display the results
blind_data['predicted_stance'] = blind_predictions

In [130]:
# Display the first few rows to confirm predictions are attached
print(blind_data[['text', 'predicted_stance']].head())

                                                text predicted_stance
0  الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحي...          Against
1     الفساد الإداري انهك البلاد ومازلنا نعتمد عل...            Favor
2  -البي بي سي ١٠ دول اوروبية توقف تطعيم اوكسفورد...          Against
3  منصة مدرستي بتسرع عملية التحول للتعليم الالكتر...            Favor
4                       انا مع تمكين المرأة اصلا URL            Favor


In [131]:
# Save the results to a new CSV file if needed
blind_data.to_csv('(Submission#2)Mawqif_AllTargets_Blind_Test_Predictions.csv', index=False)